In [ ]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
!pip install webdriver-manager
!pip install chromedriver-autoinstaller

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [664 kB]
Get:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,057 kB]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 Packages [50.5 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InR

In [ ]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
import chromedriver_autoinstaller
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import requests
import base64
import time

# Setup Chrome driver
def setup_chrome_driver():
    chromedriver_autoinstaller.install()
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    return webdriver.Chrome(options=chrome_options)

# Wait for the page to load
def wait_for_page_load(driver, timeout=30):
    WebDriverWait(driver, timeout).until(
        lambda d: d.execute_script('return document.readyState') == 'complete'
    )

# Print webpage to PDF
def print_webpage_to_pdf(driver, url, folder_path):
    try:
        driver.get(url)
        wait_for_page_load(driver)

        # Wait for a few seconds to ensure all elements are loaded
        time.sleep(5)  # Adjust the delay as needed

        # More specific JavaScript to target dynamic elements
        remove_elements_script = """
        document.querySelectorAll('your-specific-selector').forEach(el => el.remove());
        """
        driver.execute_script(remove_elements_script)

        # Generate PDF
        pdf = driver.execute_cdp_cmd("Page.printToPDF", {
            "landscape": False,
            "printBackground": True,
            "format": "A4",
            "preferCSSPageSize": True
        })
        pdf_content = base64.b64decode(pdf['data'])
        output_path = os.path.join(folder_path, f'{url.split("/")[-1]}.pdf')
        with open(output_path, 'wb') as f:
            f.write(pdf_content)
        print(f"Saved page as PDF: {url}")
    except Exception as e:
        print(f"Error saving page as PDF {url}: {e}")

# Check if link is relevant
def is_relevant_link(href):
    return href and ('html' in href or 'max' in href) and (href.startswith('http') or href.startswith('/max8/'))

# Main process
def process_website(main_url, folder_path):
    driver = setup_chrome_driver()
    os.makedirs(folder_path, exist_ok=True)

    try:
        page_response = requests.get(main_url)
        page_response.raise_for_status()
        soup = BeautifulSoup(page_response.content, 'html.parser')
        links = soup.find_all('a')

        for link in links:
            href = link.get('href')
            if is_relevant_link(href):
                full_url = href if href.startswith('http') else 'https://docs.cycling74.com' + href
                print_webpage_to_pdf(driver, full_url, folder_path)

    except requests.exceptions.RequestException as e:
        print(f"Failed to retrieve the main page: {e}")
    finally:
        driver.quit()

# Example Usage
main_url = 'https://docs.cycling74.com/max8/tutorials/05_mspbasicchapter05'
output_folder = '/content/gdrive/My Drive/MaxMSP/Basics Review'
process_website(main_url, output_folder)

Saved page as PDF: https://docs.cycling74.com/max8/tutorials/05_mspbasicchapter03
Saved page as PDF: https://docs.cycling74.com/max8/refpages/buffer~
Saved page as PDF: https://docs.cycling74.com/max8/refpages/buffer~
Saved page as PDF: https://docs.cycling74.com/max8/refpages/cycle~
Saved page as PDF: https://docs.cycling74.com/max8/refpages/line~
Saved page as PDF: https://docs.cycling74.com/max8/refpages/line~
Saved page as PDF: https://docs.cycling74.com/max8/refpages/cycle~
Saved page as PDF: https://docs.cycling74.com/max8/refpages/metro
Saved page as PDF: https://docs.cycling74.com/max8/refpages/metro
Saved page as PDF: https://docs.cycling74.com/max8/refpages/cycle~
Saved page as PDF: https://docs.cycling74.com/max8/refpages/times~
Saved page as PDF: https://docs.cycling74.com/max8/refpages/times~
Saved page as PDF: https://docs.cycling74.com/max8/refpages/patcher
Saved page as PDF: https://docs.cycling74.com/max8/refpages/buffer~
Saved page as PDF: https://docs.cycling74.com/m

In [ ]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.3 MB/s eta 0:00:00


In [ ]:
import os
import chromedriver_autoinstaller
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from PyPDF2 import PdfFileReader, PdfFileWriter
import base64
import requests
from selenium import webdriver

# Setup Chrome driver
def setup_chrome_driver():
    chromedriver_autoinstaller.install()
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    return webdriver.Chrome(options=chrome_options)

# Wait for the page to load
def wait_for_page_load(driver, timeout=30):
    WebDriverWait(driver, timeout).until(
        lambda d: d.execute_script('return document.readyState') == 'complete'
    )

# Print webpage to PDF
def print_webpage_to_pdf(driver, url, folder_path):
    try:
        driver.get(url)
        wait_for_page_load(driver)

        pdf = driver.execute_cdp_cmd("Page.printToPDF", {
            "landscape": False,
            "printBackground": True,
            "format": "A4",
            "preferCSSPageSize": True
        })
        pdf_content = base64.b64decode(pdf['data'])
        output_path = os.path.join(folder_path, f'{url.split("/")[-1]}.pdf')
        with open(output_path, 'wb') as f:
            f.write(pdf_content)
        print(f"Saved page as PDF: {url}")
    except Exception as e:
        print(f"Error saving page as PDF {url}: {e}")

# Merge PDFs
def merge_pdfs(folder_path, output_file):
    pdf_writer = PdfFileWriter()
    for item in os.listdir(folder_path):
        if item.endswith('.pdf'):
            pdf_reader = PdfFileReader(open(os.path.join(folder_path, item), 'rb'))
            for page in range(pdf_reader.getNumPages()):
                pdf_writer.addPage(pdf_reader.getPage(page))

    with open(output_file, 'wb') as out:
        pdf_writer.write(out)
    print(f"Merged PDF saved to {output_file}")

# Main process
def process_website(main_url, folder_path):
    driver = setup_chrome_driver()
    os.makedirs(folder_path, exist_ok=True)

    try:
        driver.get(main_url)
        wait_for_page_load(driver)

        links = driver.find_elements(By.TAG_NAME, "a")
        urls = [link.get_attribute('href') for link in links if link.get_attribute('href') and ('html' in link.get_attribute('href') or 'max' in link.get_attribute('href'))]

        for url in urls:
            print_webpage_to_pdf(driver, url, folder_path)

        # Merge all PDFs into one
        merge_pdfs(folder_path, os.path.join(folder_path, "merged_output.pdf"))
    except Exception as e:
        print(f"Error during processing: {e}")
    finally:
        driver.quit()

# Example Usage
main_url = 'https://docs.cycling74.com/max8/tutorials/05_mspbasicchapter05'
output_folder = '/content/gdrive/My Drive/MaxMSP/Basics Review'
process_website(main_url, output_folder)

Saved page as PDF: https://docs.cycling74.com/max8
Saved page as PDF: https://docs.cycling74.com/max8/tutorials/05_mspbasicchapter05#Exercises_in_the_fundamentals_of_MSP
Saved page as PDF: https://docs.cycling74.com/max8/tutorials/05_mspbasicchapter05#Solution_to_Exercise_2
Saved page as PDF: https://docs.cycling74.com/max8/tutorials/05_mspbasicchapter05#Solution_to_Exercise_3
Saved page as PDF: https://docs.cycling74.com/max8/tutorials/05_mspbasicchapter03
Saved page as PDF: https://docs.cycling74.com/max8/refpages/buffer~
Saved page as PDF: https://docs.cycling74.com/max8/refpages/buffer~
Saved page as PDF: https://docs.cycling74.com/max8/refpages/cycle~
Saved page as PDF: https://docs.cycling74.com/max8/refpages/line~
Saved page as PDF: https://docs.cycling74.com/max8/refpages/line~
Saved page as PDF: https://docs.cycling74.com/max8/refpages/cycle~
Saved page as PDF: https://docs.cycling74.com/max8/refpages/metro
Saved page as PDF: https://docs.cycling74.com/max8/refpages/metro
Save